# Utilize CensusReporter to Target Geographic Market Selections
## Lynn Mast
### Created for Digital Advertising APRD6342-001
#### The goal of this assignment was to gather data from CensusReporter, via an API, to identify the best geographic market targets for product expansion.

In [1]:
#Libraries
import pandas as pd
import json
import requests
from time import sleep
import numpy as np
from statistics import mean

#Data is available on github repository
msas = pd.read_csv('msas.csv')

#Tables w/info wanted
tableids = ['B19001','B01001', 'B03003']

#Join tables for json format
tableidstring = ','.join(tableids)

#A list of the CBSA codes
listCBSA = msas['CBSA'].tolist()

In [2]:
#Load all info from the URL, the sleep at the end of the loop gives pause to the API to 
#avoid overwhelming it and not getting the complete data requested.  This has a lengthy run time. 
#You can remove the hashtag in front of the #print(parsedjson) to see all the data, again, it is lengthy.
for cbsa in listCBSA:
    #print(cbsa)
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % (tableidstring, cbsa)
    #print(requesturl)
    parsedjson = json.loads(requests.get(requesturl).text)
    #print(parsedjson)   
    sleep(1) 

In [3]:
#Create list to store information from table B19001 (Household Income from $50,000 - $74,900 information) 
#and run for loop to gather information. Lengthy run time.
resultsB19001 = []

for cbsa in listCBSA:
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % ('B19001', cbsa)
    parsedjson = json.loads(requests.get(requesturl).text)
    total_hh_income = 0
    total_hh_income += parsedjson['data']['31000US%s' %(cbsa)]['B19001']['estimate']['B19001011']
    total_hh_income += parsedjson['data']['31000US%s' %(cbsa)]['B19001']['estimate']['B19001012']
    resultsB19001.append(total_hh_income)
    sleep(1)

In [4]:
#Create list to store information from table B10001 (Age information from 18 - 34 years old)
#and run for loop to gather information. Lengthy run time.
resultsB01001 = []

for cbsa in listCBSA:
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % ('B01001', cbsa)
    parsedjson = json.loads(requests.get(requesturl).text)
    total_age = 0
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001007']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001008']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001009']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001010']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001011']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001012']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001031']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001032']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001033']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001034']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001035']
    total_age += parsedjson['data']['31000US%s' %(cbsa)]['B01001']['estimate']['B01001036']
    resultsB01001.append(total_age)
    sleep(1)

In [5]:
#Create list to store information from table B03003 (Hispanic information) and run for loop to gather information. 
#Lengthy run time.
resultsB03003 = []

for cbsa in listCBSA:
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % ('B03003', cbsa)
    parsedjson = json.loads(requests.get(requesturl).text)
    total_hisp = 0
    total_hisp += parsedjson['data']['31000US%s' %(cbsa)]['B03003']['estimate']['B03003003']
    resultsB03003.append(total_hisp)
    sleep(1)

In [6]:
#Add the results of all table information back into original data frame as new columns.
msas['B01001'] = resultsB01001
msas['B19001'] = resultsB19001
msas['B03003'] = resultsB03003

In [7]:
#Find Metro Area with Max Household Income
print('Max Income')    
msas.iloc[msas.B19001.idxmax()]

Max Income


CBSA                                               35620
NAME               New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                 20320876
B01001                                       4.80694e+06
B19001                                       1.03819e+06
B03003                                        4.9944e+06
Name: 249, dtype: object

In [8]:
#Find Metro Area with Max Age
print('Max Age')  
msas.iloc[msas.B01001.idxmax()]

Max Age


CBSA                                               35620
NAME               New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                 20320876
B01001                                       4.80694e+06
B19001                                       1.03819e+06
B03003                                        4.9944e+06
Name: 249, dtype: object

In [9]:
#Find Metro Area with Max Hispanic
print('Max Hispanic') 
msas.iloc[msas.B03003.idxmax()] 

Max Hispanic


CBSA                                            31080
NAME               Los Angeles-Long Beach-Anaheim, CA
POPESTIMATE2017                              13353907
B01001                                    3.38111e+06
B19001                                         708153
B03003                                    6.03149e+06
Name: 210, dtype: object

In [11]:
#Find Metro Area with Max Percentage Across All Categories
#Add new coloumn to original data frame for each calculation performed
msas['PercentIncome'] = msas['B19001']/msas['B19001'].max()
msas['PercentAge'] = msas['B01001']/msas['B01001'].max()
msas['PercentHispanic'] = msas['B03003']/msas['B03003'].max()

#Find mean of all means
msas['MeanPercentage'] = (msas['PercentIncome'] + msas['PercentAge'] + msas['PercentHispanic'])/3

In [12]:
#Find Max Mean Percentage
print('Max Mean Percentage')
msas.iloc[msas.MeanPercentage.idxmax()]  

Max Mean Percentage


CBSA                                               35620
NAME               New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                 20320876
B01001                                       4.80694e+06
B19001                                       1.03819e+06
B03003                                        4.9944e+06
PercentIncome                                          1
PercentAge                                             1
PercentHispanic                                 0.828053
MeanPercentage                                  0.942684
Name: 249, dtype: object